In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import networkx as nx

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

## summarizers

In [ ]:
text = df["content"].iloc[0]
text

'My brother in-law (Sammy) lost his home shortly after his divorce 10 months ago. He moved in with us and brought his twin daughters (Olivia & Sloane18) with him a couple of months ago.\nHis sister (my wife) and I have one daughter (Zoey 16) and she and her cousins aren\'t close but get along fine.\n\nOlivia & Sloane have no respect for Zoey\'s privacy, none. they used to walk into her room and take everything they get their hands on. Makeup, phone accessories, clothes, school laptop etc. Zoey complained a lot and I\'ve already asked the girls to respect Zoey\'s privacy and stop taking things. My wife and Sammy saw no issue with this. After all, they\'re girls and this\'s typical teenage girls behavior. I completely disagreed. \n\nLast straw was when Zoey bought a 60$ m.a.c makeup-kit that looks like a paintset that she saved up for over a month and one of the girls, Sloane took it without permission and ruined it by mixing shades together while using it. Don\'t know much about makeup 

In [ ]:
# Initialize parser and tokenizer
parser = PlaintextParser.from_string(text, Tokenizer("english"))

# Initialize LexRank summarizer
summarizer = LexRankSummarizer()

# Get the summary
summary = summarizer(parser.document, sentences_count=3)  # Adjust the number of sentences in the summary as needed

# Print the summary
for sentence in summary:
    print(sentence)

He moved in with us and brought his twin daughters (Olivia & Sloane18) with him a couple of months ago.
I told my wife and she said she'd ask Sloane to apologize but I got Zoey a lock after I found she was moving valuable belongings out the house because of this incidence!!
I told him this's between me and my wife but she shamed me for putting a lock on Zoey's door for her cousins to see and preventing them from "spending time" with her saying I was supposed to treat them like daughters, then demanded I remove it but I said this lock does not get removed til her brother and his daughters are out of our house.


In [ ]:
# Summarize the text
summary = pyteaser.Summarize(text)

# Print the summary
for sentence in summary:
    print(sentence)

In [ ]:
def read_article(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    # Tokenize and remove stopwords
    if stopwords is None:
        stopwords = []
    sent1 = [word.lower() for word in word_tokenize(sent1) if word.lower() not in stopwords]
    sent2 = [word.lower() for word in word_tokenize(sent2) if word.lower() not in stopwords]

    # Create a set of all unique words in both sentences
    all_words = list(set(sent1 + sent2))

    # Create vectors to represent each sentence
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # Count word occurrences
    for word in sent1:
        vector1[all_words.index(word)] += 1

    for word in sent2:
        vector2[all_words.index(word)] += 1

    # Calculate cosine similarity
    return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stopwords):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    # Fill the similarity matrix with cosine similarity scores
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stopwords)

    return similarity_matrix

def generate_summary(text, top_n=5):
    # Load English stopwords
    stop_words = stopwords.words('english')

    # Tokenize the text into sentences
    sentences = read_article(text)

    # Generate the sentence similarity matrix
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)

    # Convert the similarity matrix into a graph
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)

    # Apply the PageRank algorithm to rank the sentences
    scores = nx.pagerank(sentence_similarity_graph)

    # Sort the sentences by their scores
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

    # Extract the top sentences as the summary
    summary = ' '.join([sentence for score, sentence in ranked_sentences[:top_n]])

    return summary

# Example usage:
text = df["content"].iloc[0]
summary = generate_summary(text, top_n=2)
summary

"Don't know much about makeup but that's what Zoey said when she found the kit on her bed, and was crying. Olivia & Sloane have no respect for Zoey's privacy, none."